[View in Colaboratory](https://colab.research.google.com/github/cheonbok94/DeepLearningLecture/blob/master/(ANS)RNN.ipynb)

In [0]:
#1,2 http://pytorch.org/
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision
import torch

In [0]:
# 3. authorization for using Google Drive
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# PyDrive reference:
# https://googledrive.github.io/PyDrive/docs/build/html/index.html

# 2. Create & upload a file text file.
uploaded = drive.CreateFile({'title': 'Sample upload.txt'})
uploaded.SetContentString('Sample upload file content')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

# 3. Load a file by ID and print its contents.
downloaded = drive.CreateFile({'id': uploaded.get('id')})
print('Downloaded content "{}"'.format(downloaded.GetContentString()))

Uploaded file with ID 1xtOKOMm9YrlwrIx9yj1P3O0HJVH3EDYM
Downloaded content "Sample upload file content"


In [0]:
# 4. connect to Google Drive
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass

!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

!mkdir -p drive
!google-drive-ocamlfuse drive

gpg: keybox '/tmp/tmpbhuuoysx/pubring.gpg' created
gpg: /tmp/tmpbhuuoysx/trustdb.gpg: trustdb created
gpg: key AD5F235DF639B041: public key "Launchpad PPA for Alessandro Strada" imported
gpg: Total number processed: 1
gpg:               imported: 1
··········
fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option


In [0]:
import os
import json
import pickle

In [0]:
os.listdir('./drive/4일차')

['제목 없는 문서.odt (74cd90d2)',
 '1_28 발제 준비 .odt',
 'Summarization of 2017 Deep NLP.pptx',
 '1주차.pdf',
 'Machine learning with Calculus - week1.pdf',
 '2주차.pdf',
 '3주차_선형대수 입문 기본.pdf',
 '제목 없는 문서.odt',
 '2018R1_Research',
 '새 파일 2018-03-29.pdf',
 '04_08발제준비.odt',
 '새 파일 2018-04-08.pdf',
 'Latent Constraints.pptx',
 'Fundamental Mathematical Machine learning.odt',
 '제목 없는 스프레드시트.ods',
 '1주차.pptx',
 '제목 없는 설문지의 사본.zip',
 '제목 없는 설문지.zip',
 'others',
 'lecture']

In [0]:
# 패키지들을 불러옵니다
import torch # load torch package
from torch.autograd import Variable # Load Variable Package
import torch.nn as nn  # Load Neural Network Layer
import torch.nn.functional as F # pyTorch function  불러오기  
import numpy as np # numpy 연산을 위한 package 입니다. 

In [0]:
torch.cuda.is_available() # CUDA is available 을 통해 cuda 연동 가능성 확인 

True

# Movie Sentiment Predcition

## Data Preprocessing

In [0]:
import os
import json
import pickle

저장된 데이터 위치로 가서 해당 파일을 불러옵니다.

In [0]:
with open('./drive/4일차/0_movie_300.txt') as f:
  movies = json.load(f)

In [0]:
# 샘플 구경
for i in range(12,16):
  for j in range(4):
    score = movies[i]['ratings'][j]['score']
    reple = movies[i]['ratings'][j]['reple']
    print(score,reple) # 보통의 데이터는 이렇게 우리의 자연어로 처리가 되어 있습니다. 해당 문장을 우리는 컴퓨터가 이해가능한 숫자로 표시를 해야 합니다. 

8 한글 자막이 없어, 화면으로만 즐긴 영화.. 미안해서 8점..
6 두뇌 대결이 치열하지도 않고 보고 나서 남는 것도 없었다. 그냥 게리 올드만과 케빈 베이컨의 옛 모습을 보는 것에 만족했을 뿐
8 개인적으로 재밌게 본 영화..
7 훌륭한 미장센과 배우들의 호연에 비해 아쉬운 연출력
9 또다른 시각에서의 한국전쟁. 전쟁 속 시골 마을 과부의 인생역정을 통해 한 많은 우리 민족과 역사의 슬픔을 다시금 느끼게 해 준 영화. 그리고 인간 내면의 이기심과 추악하고 더러운 모습에 또 한번 놀라다. 연말에 또 한편의 보석같은 한국영화를 찾았다!
10 가슴 아픈 영화,,
10 어렸을때 봤지만 크게 감명받은 영화
9 개봉당시  대학생이었는데   영화가  너무  슬퍼서 한동안  자리에서 앉아  있었던 기억이  나네요..
10 요즘 이상한 영화들보다  100배 재미있다.
10 최고의 변종 전쟁영화. 쓰리 킹즈는 이 영화에 빚을 졌다
10 이제 껀 본 영화 들중의 수작
8 최고의 전쟁영화 중 하나...전쟁영화라기 보다 유쾌한 도적들의 영화? 역시 돈이 최고...^^
10 나쁘진않다 딱 그정도이다.
9 볼만하다  아 파란만장한 아프리카의 여인이여
10 웬만한 영화보다 훨 나으네요~ 51년작 치고는 매우 수작임
2 그렇게 좋게평가될 영화는아닌듯 일단 전체적전개가 그렇게 자연스럽지가않음


In [0]:
# 전처리 후의 데이터
with open('./drive/4일차/data3.pkl','rb') as f:
  i2v = pickle.load(f)
  v2i = pickle.load(f)
  tr_X = pickle.load(f)
  tr_y = pickle.load(f)
  t_X = pickle.load(f)
  t_y = pickle.load(f)

In [0]:
# dictionaries 우리는 다음과 같이 index -> word로 가는 dictonary 와 vocabulary에서 index로 가는 dictonary 를 둘 다 생성해야 합니다. 이렇게 해야 우리는 컴퓨터가 이해할수 있는 숫자로 보내주는 것 , 사람이 이해할 수 있는 결과로 보여주도록 하는 두가지 
# Mapping을 사용하게 됩니다. 
i2v # 숫자에서 단어로
v2i # 단어에서 숫자로 밑에 보이는 output이 v2i의 결과 입니다. 

{'<PAD>': 0,
 '.': 1,
 '이': 2,
 '영화': 3,
 '다': 4,
 '가': 5,
 '..': 6,
 '는': 7,
 '에': 8,
 '을': 9,
 '도': 10,
 '의': 11,
 '은': 12,
 '...': 13,
 '들': 14,
 '고': 15,
 '한': 16,
 '만': 17,
 ',': 18,
 '를': 19,
 '요': 20,
 '너무': 21,
 '음': 22,
 '좋': 23,
 '게': 24,
 '지': 25,
 '서': 26,
 '것': 27,
 '나': 28,
 '는데': 29,
 '봤': 30,
 '스토리': 31,
 '적': 32,
 '연기': 33,
 '?': 34,
 '거': 35,
 '듯': 36,
 '으로': 37,
 '로': 38,
 '점': 39,
 '했': 40,
 '습니다': 41,
 '볼': 42,
 '진짜': 43,
 '았': 44,
 '어요': 45,
 '지만': 46,
 '정말': 47,
 '배우': 48,
 '데': 49,
 '안': 50,
 '과': 51,
 '그냥': 52,
 '좀': 53,
 '내': 54,
 '!': 55,
 '보고': 56,
 '인': 57,
 '에서': 58,
 '하고': 59,
 '~': 60,
 '수': 61,
 '내용': 62,
 '못': 63,
 '평점': 64,
 '잘': 65,
 '보다': 66,
 '던': 67,
 '재미': 68,
 '와': 69,
 '아': 70,
 '....': 71,
 '네요': 72,
 '그': 73,
 '보는': 74,
 '더': 75,
 '기대': 76,
 '있': 77,
 '였': 78,
 '아니': 79,
 '라': 80,
 '액션': 81,
 '네': 82,
 '면': 83,
 '생각': 84,
 '하는': 85,
 '사람': 86,
 '본': 87,
 '1': 88,
 '느낌': 89,
 '감동': 90,
 '없는': 91,
 'ㅋㅋ': 92,
 '별로': 93,
 '재미없': 94,
 ';': 95,
 '시

In [0]:
# input data
tr_X[70] # 우리의 학습 데이터는 이렇게 문장이 아닌 숫자로 표현이 되게 됩니다. 

[98,
 2,
 1033,
 2387,
 247,
 2,
 3,
 19,
 74,
 27,
 12,
 1416,
 19999,
 11,
 96,
 66,
 439,
 96,
 1250]

In [0]:
# index list to word list
[i2v[word] for word in tr_X[70]] # 해당 문장을 다시 우리가 이해가능한 문자의 세계로 보내볼까요? 

['말',
 '이',
 '필요없',
 '슴',
 '~~',
 '이',
 '영화',
 '를',
 '보는',
 '것',
 '은',
 '똥',
 '<UNK>',
 '의',
 '시간',
 '보다',
 '아까운',
 '시간',
 '이에요']

In [0]:
tr_y[79]

0.7777777777777778

In [0]:
len(tr_X)

283365

In [0]:
len(t_X)

31485

In [0]:
len(tr_X[0::10])

28337

#### **Data Preprocessing Summary**

i2v : index to vocabulary 해당 되는 index 에 대한 단어의 Dictonary 

v2i : vocabulary to inde 해당 되는 단어에 대한 index dictonary 

tr_X : training Data 파일 X (문장갯수 x 문장 단어)

tr_y: training Data 파일 y  (문장갯수 y scale value : 1-10 scale : 1 ->0,  6 - > 0.55556(5/9) , 10 ->1)

In [0]:
def new_data(x,y,max_len = 10,min_len = 6):
	keep_x = []
	keep_y = []
	for i,data in enumerate(x) :
		if len(data) <= max_len and len(data) >= min_len:
			keep_x.append(data)
			keep_y.append(y[i])
	return keep_x,keep_y
	

In [0]:
simple_x,simple_y = new_data(tr_X,tr_y)
simple_tx,simple_ty = new_data(t_X,t_y)

In [0]:
len(simple_x)

64513

### Define Model

### Batch 를 구성해보겠습니다. 
Q : 우리의 문장에서 항상 단어의 갯수는 일정한가요?

문제점: Tensor로 변환하기 위해서는 추가 처리 필요 . eg. Sentence1 : W1 , W2, W3 ,W4 / Sentence : W1 , W2 ,W3 ->  2*3이 맞나요? 2*4가 맞나요? 

해결법 : Batch * max_length  Tensor로 변환합니다. eg. 2*4의 경우 2 번째 문장의 마지막에 PAD_token을 처리해줍니다. Index 보통 : 0 

In [0]:
def makeBatch(batch_sequences, max_len=100):
    batch_sequences = [torch.Tensor(sequence) for sequence in batch_sequences] # Batch 에서 각 Sequence 저장 
    lengths = [len(sequence) for sequence in batch_sequences] # Max Length 처리 
    output = torch.zeros(len(batch_sequences), max_len) # Batch * Max_length Tensor 생성 
    for i, sequence in enumerate(batch_sequences): 
        length = lengths[i]
        output[i, :length] = sequence[:length] # 생성된 Input Tensor에 해당 값을 넣어줍니다. 
    output = output.long() # 정수 값을 사용할 때는 pytorch에서는 .long() 을 사용합니다.  int()는 사용할 수 없습니다. 
    return output


## Training Step

In [0]:
import random
class DataLoader(object):
    def __init__(self,data,batch_size,shuffle = True,training= True ):
        self.src_data = data[0]
        self.trg_data = data[1]
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.iteration = 0
        self.max_index = int(len(self.src_data)/batch_size)
        if training :
            self.init_epoch()
    def __iter__(self):
      return self
    def len_data(self):
        return len(self.src_data)
    def init_epoch(self):
        paired_data = list(zip(self.src_data,self.trg_data))
        random.shuffle(paired_data)
        self.src_data,self.trg_data = zip(*paired_data)
        self.iteration =0
    def __next__(self):
        return self.next()
    def next(self):
        while True:
            if self.iteration > self.max_index:
                self.init_epoch()
                raise StopIteration
                
            selected_src = self.src_data[self.iteration*self.batch_size:(self.iteration+1)*self.batch_size]
            len_src = [len(s) for s in selected_src]
            selected_src = makeBatch(selected_src,max_len = max(len_src))
            selected_trg = self.trg_data[self.iteration*self.batch_size:(self.iteration+1)*self.batch_size]
            selected_trg = torch.Tensor(selected_trg)
            self.iteration += 1
            
            return (selected_src,selected_trg)
     
    

In [0]:
trainDataLoader = DataLoader((tr_X,tr_y),32,shuffle =True,training =True )
testDataLoader = DataLoader((t_X,t_y),32,shuffle = False ,training =False)
sim_trainDataLoader = DataLoader((simple_x,simple_y),32,shuffle =True,training =True )  # 데이터가 너무 많아 데이터의 수를 줄여서 사용하기 위해 simple data를 사용해봅니다.
sim_testDataLoader = DataLoader((simple_tx[0::10],simple_ty[0::10]),32,shuffle = False ,training =False)

In [0]:
def train_model (model,dataloader,criterion,optimizer,epoch=4):
    step = 0
    print ("[+] Training Start ")
    model.train()
    for e in range(epoch):
        cnt = 0.0
        total_loss = 0.0
        
        for (i,data) in enumerate(dataloader):
                x = Variable(data[0]).cuda()
                y = Variable(data[1]).cuda()
                optimizer.zero_grad()
                outputs = model(x)
                loss = criterion(outputs,y)
                loss.backward()
                optimizer.step()
                total_loss += loss.data[0]

                step+=1
        print("[+] End of epoch %d avg_loss : %1.3f" %(e+1,total_loss/step))
        step = 0

In [0]:
def test_model (model,dataloader,criterion):
		step = 0
		model.eval()
		total_loss = 0.0 
		for (i,data) in enumerate(dataloader):
			x = Variable(data[0]).cuda()
			y = Variable(data[1]).cuda()
							 
			outputs = model(x)
			loss = criterion(outputs,y)
			total_loss += loss.data[0]
								
			step+=1
		print ("Total avg test loss %1.3f" %(total_loss/step))

In [0]:
sim_trainDataLoader.max_index

885

##  모델을 만들어 봅시다. 

embedding_dim = 256 
hidden_size = 256
num layer = 1 
bidircetioanl = False 
Adam Optimizer 
learning rate = 0.001
Loss MSELoss

In [0]:
class BiRNN(nn.Module):
    def __init__(self,vocab_size,embedding_dim,hidden_size,num_layers,bidirectional):
        super(BiRNN,self).__init__()
        self.embedding = nn.Embedding(vocab_size,embedding_dim)
        self.rnn = nn.RNN(input_size=embedding_dim, hidden_size=hidden_size,  # Rnn의 인자 input_size, hidden_size, num_layers = number of layers , batch_first,bidirectional = Bidicrectonal RNN 사용 
                        num_layers=num_layers, batch_first=True, bidirectional=bidirectional)
        if self.rnn.bidirectional:
            self.W_out = nn.Linear(hidden_size*2,1) # Bidirectional RNN 의 경우 한 방향에 대한 hidden diension 이 2개가 있으므로 *2를 하여 사용합니다. 
        else:
            self.W_out = nn.Linear(hidden_size,1)

    def forward(self,x):
        b,seq = x.shape
        x = self.embedding(x)
        outputs, h = self.rnn(x) # RNN input을 넣어줍니다. [b, seq, hid]
        outputs = outputs.sum(1) # Sentence Reprensentation 만들기 : rnn outputs (Batch, L,hid) 값들 합치기를 합을(Length) 합니다.[b, hid] -
        outputs = self.W_out(outputs) # [b, 1]
        return F.sigmoid(outputs.view(b))

In [0]:
vocab_size = len(i2v)

reg = BiRNN(vocab_size = vocab_size,embedding_dim = 256,hidden_size = 512 ,num_layers = 1 ,bidirectional = False)
reg = reg.cuda()
criterion = nn.MSELoss()
opt = torch.optim.Adam(reg.parameters(),lr= 0.001)

In [0]:
train_model(reg,sim_trainDataLoader,criterion,opt)

[+] Training Start 
[+] End of epoch 1 avg_loss : 0.068
[+] End of epoch 2 avg_loss : 0.052
[+] End of epoch 3 avg_loss : 0.044
[+] End of epoch 4 avg_loss : 0.037


In [0]:
test_model(reg,sim_testDataLoader,criterion)

Total avg test loss 0.058


### 일방향 RNN의 한계  -> 왼쪽 단어만 보고 해당 정보를 Encoding ! -> 양방향을 보면 어떨까? --> Bidirectional RNN

In [0]:
reg = BiRNN(vocab_size = vocab_size,embedding_dim = 256,hidden_size = 256 ,num_layers = 1 ,bidirectional = True)
reg = reg.cuda()
criterion = nn.MSELoss()
opt = torch.optim.Adam(reg.parameters(),lr= 0.001)

In [0]:
train_model(reg,sim_trainDataLoader,criterion,opt)

[+] Training Start 
[+] End of epoch 1 avg_loss : 0.073
[+] End of epoch 2 avg_loss : 0.051
[+] End of epoch 3 avg_loss : 0.042
[+] End of epoch 4 avg_loss : 0.035


In [0]:
test_model(reg,sim_testDataLoader,criterion)

Total avg test loss 0.061


### RNN 보다 Long Sentence에도 학습을 잘하도록 모델을 만들어보자 -> GRU 로 바꿔보자 

In [0]:
class BiRNN(nn.Module):
    def __init__(self,vocab_size,embedding_dim,hidden_size,num_layers,bidirectional):
        super(BiRNN,self).__init__()
        self.embedding = nn.Embedding(vocab_size,embedding_dim)
        self.rnn = nn.GRU(input_size=embedding_dim, hidden_size=hidden_size,  # Rnn의 인자 input_size, hidden_size, num_layers = number of layers , batch_first,bidirectional = Bidicrectonal RNN 사용 
                        num_layers=num_layers, batch_first=True, bidirectional=bidirectional)
        if self.rnn.bidirectional: 
            self.W_out = nn.Linear(hidden_size*2,1) # Bidirectional RNN 의 경우 한 방향에 대한 hidden diension 이 2개가 있으므로 *2를 하여 사용합니다. 
        else:
            self.W_out = nn.Linear(hidden_size,1)

    def forward(self,x):
        b,seq = x.shape
        x = self.embedding(x)
        outputs, h = self.rnn(x) # RNN input을 넣어줍니다. [b, seq, hid]
        outputs = outputs.sum(1) # Sentence Reprensentation 만들기 : rnn outputs (Batch, L,hid) 값들 합치기를 합을(Length) 합니다.[b, hid] -
        outputs = self.W_out(outputs) # [b, 1]
        return F.sigmoid(outputs.view(b))

In [0]:
reg = BiRNN(vocab_size = vocab_size,embedding_dim = 256,hidden_size = 512 ,num_layers = 1 ,bidirectional = False) # Model을 선언해 봅시다. 
reg = reg.cuda() # cuda 올리기 
criterion = nn.MSELoss() # MSE LOSS 선언 
opt = torch.optim.Adam(reg.parameters(),lr= 0.001) # OPTIMIZER 부르기 

In [0]:
train_model(reg,sim_trainDataLoader,criterion,opt)

[+] Training Start 
[+] End of epoch 1 avg_loss : 0.068
[+] End of epoch 2 avg_loss : 0.052
[+] End of epoch 3 avg_loss : 0.044
[+] End of epoch 4 avg_loss : 0.036


In [0]:
test_model(reg,sim_testDataLoader,criterion)

Total avg test loss 0.064


###  이번에도 GRU 에 Bidirectional 을 추가하자.

In [0]:
reg = BiRNN(vocab_size = vocab_size,embedding_dim = 256,hidden_size = 256 ,num_layers = 1 ,bidirectional = True) # Bidirectonal 옵션을 바꿔 줍시다. 
reg = reg.cuda()
criterion = nn.MSELoss()
opt = torch.optim.Adam(reg.parameters(),lr= 0.001)

In [0]:
train_model(reg,sim_trainDataLoader,criterion,opt)

[+] Training Start 
[+] End of epoch 1 avg_loss : 0.070
[+] End of epoch 2 avg_loss : 0.051
[+] End of epoch 3 avg_loss : 0.043
[+] End of epoch 4 avg_loss : 0.035


In [0]:
test_model(reg,sim_testDataLoader,criterion)

Total avg test loss 0.125


### 이번에는 더 깊은 Layer 로 변화해보겠습니다. 

In [0]:
reg = BiRNN(vocab_size = vocab_size,embedding_dim = 256,hidden_size = 256 ,num_layers = 2 ,bidirectional = True) # Bidirectional 로 바꿔 볼까요? hidden_size는 1/2로 주는게 좋아요 
reg = reg.cuda()
criterion = nn.MSELoss()
opt = torch.optim.Adam(reg.parameters(),lr= 0.001)

In [0]:
train_model(reg,sim_trainDataLoader,criterion,opt)

[+] Training Start 
[+] End of epoch 1 avg_loss : 0.065
[+] End of epoch 2 avg_loss : 0.046
[+] End of epoch 3 avg_loss : 0.037
[+] End of epoch 4 avg_loss : 0.029


In [0]:
test_model(reg,sim_testDataLoader,criterion)

Total avg test loss 0.050


## Drop out option을 추가해보자 

In [0]:
class BiRNN(nn.Module):
    def __init__(self,vocab_size,embedding_dim,hidden_size,num_layers,bidirectional):
        super(BiRNN,self).__init__()
        self.embedding = nn.Embedding(vocab_size,embedding_dim)
        self.rnn = nn.GRU(input_size=embedding_dim, hidden_size=hidden_size,  # Rnn의 인자 input_size, hidden_size, num_layers = number of layers , batch_first,bidirectional = Bidicrectonal RNN 사용 
                        num_layers=num_layers, batch_first=True, bidirectional=bidirectional,dropout = 0.2) # 주의 Drop out이 어디 값이 될까요? + Drop out 의 숫자는 유지일까요? 그만큼 drop 시키는 것일까요? # training /test time의 차이는 무엇일까요?
        if self.rnn.bidirectional: 
            self.W_out = nn.Linear(hidden_size*2,1) # Bidirectional RNN 의 경우 한 방향에 대한 hidden diension 이 2개가 있으므로 *2를 하여 사용합니다. 
        else:
            self.W_out = nn.Linear(hidden_size,1)

    def forward(self,x):
        b,seq = x.shape
        x = self.embedding(x)
        outputs, h = self.rnn(x) # RNN input을 넣어줍니다. [b, seq, hid]
        outputs = outputs.sum(1) # Sentence Reprensentation 만들기 : rnn outputs (Batch, L,hid) 값들 합치기를 합을(Length) 합니다.[b, hid] -
        outputs = self.W_out(outputs) # [b, 1]
        return F.sigmoid(outputs.view(b))

In [0]:
reg = BiRNN(vocab_size = vocab_size,embedding_dim = 256,hidden_size = 256 ,num_layers = 2 ,bidirectional = True) # Bidirectional 로 바꿔 볼까요? hidden_size는 1/2로 주는게 좋아요 
reg = reg.cuda()
criterion = nn.MSELoss()
opt = torch.optim.Adam(reg.parameters(),lr= 0.001)

In [0]:
train_model(reg,sim_trainDataLoader,criterion,opt)

[+] Training Start 
[+] End of epoch 1 avg_loss : 0.068
[+] End of epoch 2 avg_loss : 0.047
[+] End of epoch 3 avg_loss : 0.038
[+] End of epoch 4 avg_loss : 0.031


In [0]:
test_model(reg,sim_testDataLoader,criterion)

Total avg test loss 0.054


## CNN 기반의 Sentiment Analysis Model (Practive : 그림을 그리면서 코드를 짜보는 연습.)

In [0]:
class CNNReg(nn.Module):
    def __init__(self, voca_size, embedding_dim):
        super(CNNReg, self).__init__()
        self.kernel_size = [2, 3, 4, 5] # CNN -- Window Size 다르게?  Why ? 다양한 Phrase 를 잡는다고 생각 하시면 됩니다. Phrase 는 뭔가요? European Union 이걸 우린 하나 하나로 인식하지 않고 두 글자를 하나의 의미로 인식합니다. 이런 단위가 Phrase입니다.
                                        # 이런 Phrase 가 다양하게 Capture 해내기 위해서 우린 다양한 Convolutuin size 를 정하고 사용합니다.
        self.channel_out = 10
        self.embedding = nn.Embedding(voca_size, embedding_dim) # Embedding 을 정의 합니다.
        self.conv1 = nn.ModuleList(
            [nn.Conv2d(1, self.channel_out, (k, embedding_dim)) for k in self.kernel_size]) # Q1)한번 kernel size를 다양하게 만들어볼까요? Q2) 그냥 List 로 하면 안되나요?  안 됩니다.. 그럼 그 안에 있는 애들은 gradient 가 안흐르게 됩니다.  
        self.linear1 = nn.Linear(self.channel_out*len(self.kernel_size), 10) #   Channel_out * len(self.kernel_size )-Max Pooling 했기 때문입니다. -> 10 
        self.linear2 = nn.Linear(10, 1) # 10 -> 1 Fully Connected Network 를 정의해봅시다. 
        self.dropout = nn.Dropout() # Drop_out layer 를 정의 해봅니다. default p = 0.5

    def forward(self, x):
        embed = self.embedding(x)   # (N, L, D)
        embed = embed.unsqueeze(1)  # (N,1,L,D), 1: channel_in

        # [(N,Channel_out,W,1), ...] * len(Kernel_size)
        feature_maps = [F.relu(conv(embed)) for conv in self.conv1] # self.conv1 에 있는 데이터 들을 하나씩 불러오고 그 conv를 각각 적용해봅니다.  -> # N,C,X,1

        # [(N,Channel_out,W), ...] * len(Kernel_size)
        feature_maps = [feature_map.squeeze(3) for feature_map in feature_maps] # N,1,X,1 -> N,1,X (X인 이유는 Kernel size 마다 다른 결과가 나오기 때문입니다.)
        
        # [(N, Channel_out), ...] * len(Kernel_size)
        pooled_output = [F.max_pool1d(feature_map, feature_map.size(2)) for feature_map in feature_maps] # Max Pooling 을 수행합니다. 
        output = torch.cat(pooled_output, 1) # pooling 된 결과를 column 에 붙입니다. 
        output = output.view(output.size(0), -1) # ??
        output = self.dropout(output) # Drop out 을 적용해줍니다. (성능을 높히기 위해)
        output = F.relu(self.linear1(output)) # fully connected 1  
        output = self.dropout(output)  # drop out 
        output = self.linear2(output) # fully connected 2 
        return output

In [0]:
reg = CNNReg(voca_size = vocab_size,embedding_dim = 256) # CNN기반의 모델을 불러옵니다. 
reg = reg.cuda()
criterion = nn.MSELoss()
opt = torch.optim.Adam(reg.parameters(),lr= 0.001)

In [0]:
train_model(reg,sim_trainDataLoader,criterion,opt)

[+] Training Start 
[+] End of epoch 1 avg_loss : 0.061
[+] End of epoch 2 avg_loss : 0.058
[+] End of epoch 3 avg_loss : 0.056
[+] End of epoch 4 avg_loss : 0.055


In [0]:
test_model(reg,sim_testDataLoader,criterion)

Total avg test loss 0.053


### Pytorch 연습 코드들

tutorial : https://github.com/yunjey/pytorch-tutorial 

PYTORCH Template : https://github.com/victoresque/pytorch-template 

Advanced 

Pytorch nmt : https://github.com/OpenNMT/OpenNMT-py 

Seq2seq  : https://github.com/IBM/pytorch-seq2seq 

Awesome Pytorch list 

https://github.com/bharathgs/Awesome-pytorch-list




### model SAVE

In [0]:
torch.save(reg.state_dict(), 'rnn_regression.pkl')